# <ins> Finalizing the boundary-setting </ins>
Based on the last three models, we managed to obtain a pretty solid mapping of the areas of Eindhoven which could properly fit the definitions provided by Fieke.
It is crucial now to define these boundaries as accurately as possible; the goal of this notebook is thus to see how we can overlay Alex and Sjoerd's findings to the current mappings, and extract a more detailed map with clearly defined boundaries.


## Overlaying the maps
First, as I mentioned, we need to overlay the final mapping of Eindhoven that I designed to the mapping provided by Alex and Sjoerd; this will give us the most accurate mapping of the zones we will be working with, and will allow us to properly model the points of connection we want to place later on.

Analyzing the map the Built Environment guys provided, you can see that **all of the regions fit inside the boundary-passing model zones!** This means that what we need to do at this point is to merge and fit the specific OSM boundaries we will use to these distinctions, and create a workable map where all of the regions are split in this manner. The most accurate manner of doing this would be to manually extract the polygons using GeoJSON or by using QGIS and referencing the two maps; I opted to use the first approach, since we then can easily re-shape the neighborhoods if new data is provided.

You can see the file containing these boundaries **refinedMap.geojson** in the same directory as this Python notebook; after loading using folium and geopandas, it looks as follows:




In [2]:
import geopandas as gpd
import osmnx as ox
import json
import folium
from IPython.display import display, HTML

with open("refinedMap.geojson") as openedFile:
    polygonGeoJSON = json.load(openedFile)

# Convert GeoJSON to GeoDataFrame
geoDataEindhoven = gpd.GeoDataFrame.from_features(polygonGeoJSON["features"])
polygon = geoDataEindhoven.geometry.values[0]

# Create a Folium map centered on Eindhoven
eindhoven_map = folium.Map(location=[51.4416, 5.4697], zoom_start=12, tiles="cartodbpositron")

# Add the GeoJSON data to the map
folium.GeoJson(
    polygonGeoJSON,
    name="Eindhoven Boundaries",
    style_function=lambda feature: {
        "fillColor": "blue",
        "color": "black",
        "weight": 1,
        "fillOpacity": 0.5,
    },
).add_to(eindhoven_map)
display(HTML(eindhoven_map._repr_html_()))

Now, we convert the GeoJSON data that I plotted into a working polygon, so that we may use it with OSMNx for optimal point plotting.

In [3]:
import geopandas as gpd
import pandas as pd
import osmnx as ox
import json

with open("refinedMap.geojson") as openedFile:
    polygonGeoJSON = json.load(openedFile)
    
geoDataEindhoven = gpd.GeoDataFrame.from_features(polygonGeoJSON["features"])
polygon = geoDataEindhoven.geometry.values[0]


Everything's converted - we can move on to finding the points we will use for our hubs.

## Finding low-efficiency zones

I am still brainstorming if there is a better way of going about this, but for now, the best approach would be to do what we presented in the midterm presentation: highlighting low-scoring zones in terms of 15-minute walkability, and then further refining where within these zones it would be best to place the bikes.

In [4]:
from shapely.geometry import Point, Polygon
import numpy as npo
import folium

with open("refinedMap.geojson") as openedFile:
    polygonGeoJSON = json.load(openedFile)

geoDataEindhoven = gpd.GeoDataFrame.from_features(polygonGeoJSON["features"])
polygons = geoDataEindhoven.geometry

# I use the same tags I used before, since we agreed on them as a form of baseline, and the map Sjoerd showed used very
# similar tags. I added a few things to the "leisure" and "shop" tags because I realized that they are not as varied
# as the "office" and "amenity" tags. 

tags = {
    'office': ['yes'],  # All workplaces, since OSM has an incredible amount of identifiers.
    'shop': ['supermarket', 'convenience', 'bakery', 'greengrocer', 'butcher'],
    'amenity': [
        'pharmacy', 'hospital', 'clinic', 'doctors', 
        'school', 'kindergarten', 'college', 'university',  
        'cafe', 'restaurant', 'bar', 'cinema', 'theatre',  
        'community_centre', 'library', 'bicycle_rental',
        'place_of_worship'
        # Place of worship can be considered "key" ammenity for some, so should be included.
    ],
    'leisure': [
        'park', 'fitness_centre', 'sports_centre', 'stadium', 
        'dog_park', 'pitch', 'swimming_pool', 'playground',
        'nature_reserve'
    ],
    'shop' : [
        'supermarket', 'convenience', 'bakery', 'greengrocer', 'butcher',
        'department_store', 'general', 'cosmetics', 'stationery'
    ]
}

# First, we generate a grid of poinst so that this may actually be treated as a graphing problem.
def generateGrid(polygon, spacing=0.0015):
    minx, miny, maxx, maxy = polygon.bounds
    x_coords = npo.arange(minx, maxx, spacing)
    y_coords = npo.arange(miny, maxy, spacing)
    points = [Point(x,y) for x in x_coords for y in y_coords if polygon.contains(Point(x,y))]
    return points

# Then, we may simply analyze the polygons the same way we did way back in the original simulations,
# and give each polygon a score according to its accessibility.
lowScoreZones = []
for polygon in polygons:
    # As usual, needs some debugging messages
    print("Processing a new polygon... as always, might take a while.")
    
    # Generate a grid of points within the polygon - this is based on size of polygon, so 
    # there are obviously more points in larger polygons. Smallest ones has ~15, largest ~67
    grid_points = generateGrid(polygon)
    print(f"Generated {len(grid_points)} grid points within the polygon.")
    
    # Query OSM for the specified tags
    amenities = ox.features_from_polygon(polygon, tags)
    print(f"Found {len(amenities)} amenities in the polygon.")
    
    # Check accessibility for each grid point
    lowScoringPoints = []
    amenityCountsPer = []
    
    for point in grid_points:
        # Create a 15-minute walking isochrone (approximately 1.2 km radius)
        isochrone = point.buffer(0.012)  # 1.2 km in degrees
        
        # Check if the isochrone contains any amenities
        accessible_amenities = amenities[amenities.intersects(isochrone)]
        amenityCount = len(accessible_amenities)
        amenityCountsPer.append(amenityCount)
        
    # The rubric used by CityAccessMap scores the "Average" zones to be below 50% of the max;
    # here, I use 30%.
    maxAm = max(amenityCountsPer) if amenityCountsPer else 0
    threshold = maxAm * 0.5
    
    for point, count in zip(grid_points, amenityCountsPer):
        if count < threshold:
            lowScoringPoints.append(point)
    
    print(f"Identified {len(lowScoringPoints)} low-scoring points in the polygon.")
    lowScoreZones.append(gpd.GeoDataFrame(geometry=lowScoringPoints))
    
# Now, we need to visualize this! 
# We do what we did earlier, and convert to a single GeoDataFrame.
lowScoreZonesGDF = gpd.GeoDataFrame(pd.concat(lowScoreZones, ignore_index=True))
eindhoven_map = folium.Map(location=[51.4416, 5.4697], zoom_start=12, tiles="cartodbpositron")

# Add the low-scoring points to the map
for _, row in lowScoreZonesGDF.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=4,  # Slightly larger radius for low-scoring points
        color="red",  # Red color for low-scoring points
        fill=True,
        fill_opacity=0.8,
    ).add_to(eindhoven_map)

# Add the generated grid points to the map
for polygon in polygons:
    # Generate a grid of points within the polygon
    grid_points = generateGrid(polygon)
    
    # Add each grid point to the map
    for point in grid_points:
        folium.CircleMarker(
            location=[point.y, point.x],
            radius=2,  # Smaller radius for grid points
            color="green",  # Green color for grid points
            fill=True,
            fill_opacity=0.6,
        ).add_to(eindhoven_map)

# Query OSM for bus stops within the Eindhoven boundary
bus_stop_tag = {'highway': 'bus_stop'}
bus_stops = ox.features_from_polygon(polygons.unary_union, bus_stop_tag)  # Combine all polygons into one

# Debugging: Print the number of bus stops found
print(f"Found {len(bus_stops)} bus stops in Eindhoven.")

# Add bus stops to the map
for _, bus_stop in bus_stops.iterrows():
    if bus_stop.geometry.geom_type == 'Point':  # Ensure the geometry is a point
        folium.CircleMarker(
            location=[bus_stop.geometry.y, bus_stop.geometry.x],
            radius=3,  # Smaller radius for bus stops
            color="blue",  # Blue color for bus stops
            fill=True,
            fill_opacity=0.8,
            tooltip="Bus Stop",  # Optional: Add a tooltip
        ).add_to(eindhoven_map)

# Add the original polygons
folium.GeoJson(
    polygonGeoJSON,
    name="Eindhoven Boundaries",
    style_function=lambda feature: {
        "fillColor": "blue",
        "color": "black",
        "weight": 1,
        "fillOpacity": 0.2,
    },
).add_to(eindhoven_map)

# Save the map (for sharing)
output_file = "eindhoven_map_with_bus_stops.html"
eindhoven_map.save(output_file)
print(f'Map saved to {output_file}')

# Display the map
eindhoven_map

Processing a new polygon... as always, might take a while.
Generated 640 grid points within the polygon.
Found 358 amenities in the polygon.
Identified 198 low-scoring points in the polygon.
Processing a new polygon... as always, might take a while.
Generated 499 grid points within the polygon.
Found 571 amenities in the polygon.
Identified 283 low-scoring points in the polygon.
Processing a new polygon... as always, might take a while.
Generated 229 grid points within the polygon.
Found 103 amenities in the polygon.
Identified 32 low-scoring points in the polygon.
Processing a new polygon... as always, might take a while.
Generated 254 grid points within the polygon.
Found 172 amenities in the polygon.
Identified 27 low-scoring points in the polygon.
Processing a new polygon... as always, might take a while.
Generated 183 grid points within the polygon.
Found 67 amenities in the polygon.
Identified 30 low-scoring points in the polygon.


/var/folders/j9/whgjbhdn4g3fntrqmmjz7p2r0000gn/T/ipykernel_15156/464915959.py:118: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  bus_stops = ox.features_from_polygon(polygons.unary_union, bus_stop_tag)  # Combine all polygons into one


Found 275 bus stops in Eindhoven.
Map saved to eindhoven_map_with_bus_stops.html


## Mapping the improvement zones.
Looking at the map displayed above, we can see clearly see a number of zones which acommodate access to fewer services than is the average for the rest of the city. We specifically use this criterium for measurement, since we have already checked for 15-minute accessibility, alongside optimal population density and service diversity; the goal now is to "even out" these zones which do not adhere to a city-wide standard of mobility.

Sjoerd and Alex found a number of suitable locations which may be used for the placement of mobility hubs around the city. I overlaid these maps to obtain a provisional selection, manually mapping the potential zones of placement in accordance with existing infrastructure that I found on Google Maps. Some explanations are in order to fully explain the mapping of these locations:

1) To make our project future-proof, we decided to plan with the transformations to the Eindhoven Station Area in mind, specifically adhering to the planned modifications to the Eindhoven Central station as planned by KCAP. Although the placement of the hub in a zone of high-mobility may initially seem counterintuitive, after deliberation we have concluded that such an incusion would seamlessly integrate with the existing transport infrastructure and allow for further access to other mobility hubs.

2) The specific placement of the hubs as *physical* spaces is done as follows:
- Where possible 
- Where possible, the zones are incorporated into preexisting infrastructure, replacing vacant lots where possible, and where this is impossible, being composed into existing car-centric infrastructure, such as parking lots.
- If the above is not possible, the zones are extended to give planners an array of potential locations which can still be modeled appropriately

    In general, the zones are mapped in a manner such that they can be abstracted to a single point consistent with the grid used for previous mapping.

Some more comments regarding specific zones:
1) Mercuriuslaan zone: fitted into a pre-existing green space.
2) Catharina Ziekhuis / Maxima Medisch Centrum / Anna Ziekhuis: accessibility of the disabled
3) Catharina Ziekhuis: big transit point.
3) Next to Stroomz de Akkers: playground
4) Lavendelstraat: tight area with slight expansion and green zone.
5) Solmsweg: vacant lot between two apartment blocks; cant use other one

You may see the zones in the Folium map provided below: they are highlighted in a XXX color so that they may be more distinctly recognized. In the next step, we will try to see how these may improve communication to acquire an estimate of their sustainability.

In [5]:
with open("hubLocations.geojson") as hubMapFile:
    hubPolygon = json.load(hubMapFile)

# As with the full Eindhoven map, we convert the GeoJSON to a GeoDataFrame:
hubGeoDataFrame = gpd.GeoDataFrame.from_features(hubPolygon["features"])
polygon = hubGeoDataFrame.geometry.values[0]

# Create a Folium map centered on the hub locations drawn in GeoJSON:
hubMapEindhoven = folium.Map(location=[51.4416, 5.4697], zoom_start=12, tiles="cartodbpositron")

# We want to highlight the placements of the mobility hubs, so we specify their coordinate points as
# geojson features. This was A PAIN to write down, since gejson formatting is finicky.
hubCoordinates = [
    [
        [5.479242878818951, 51.44325775636963],
        [5.4776044181902535, 51.443029567735266],
        [5.478054544736551, 51.44202702055455],
        [5.479500951372074, 51.442210323735026],
        [5.479242878818951, 51.44325775636963]
    ],
    [
        [5.4705474804922005, 51.46453387794932],
        [5.4705542616922, 51.46434165458419],
        [5.471249334683762, 51.464352216328365],
        [5.471235772283819, 51.4644599459815],
        [5.473435691022786, 51.46451142198288],
        [5.473427854871261, 51.464868533654055],
        [5.47064438961209, 51.46480740637557],
        [5.470659881996994, 51.46454037572332],
        [5.4705474804922005, 51.46453387794932]
    ],
    [
        [5.4909987501103785, 51.47036299334738],
        [5.490919511068853, 51.46992304756114],
        [5.491994405882707, 51.469843642259804],
        [5.4920529738699315, 51.47017628516602],
        [5.492080535274624, 51.4702921730929],
        [5.4909987501103785, 51.47036299334738]
    ],
    [
        [5.488958986867033, 51.454195789565574],
        [5.488928429428, 51.453770525969844],
        [5.489634645789067, 51.45373455822033],
        [5.4896719937700595, 51.454187326647],
        [5.489675389041054, 51.45432696460165],
        [5.489230608544062, 51.45433754321695],
        [5.489217027460029, 51.45420848394019],
        [5.488958986867033, 51.454195789565574]
    ],
    [
        [5.474621202205924, 51.47619067373631],
        [5.474591931246323, 51.4758989748988],
        [5.475477377799393, 51.475871628037254],
        [5.475506648758994, 51.476181558175796],
        [5.474621202205924, 51.47619067373631]
    ],
    [
        [5.520386704860698, 51.439007813779085],
        [5.519824170435442, 51.438839165418216],
        [5.519984894556927, 51.438605394382506],
        [5.5204617094507, 51.43868554458689],
        [5.520560822659149, 51.438752336315815],
        [5.520386704860698, 51.439007813779085]
    ],
    [
        [5.488345696419913, 51.420028453490715],
        [5.488251180343866, 51.419494880781485],
        [5.490116629225724, 51.41935218004775],
        [5.4902260688930085, 51.41989506089749],
        [5.489972367845638, 51.41999432984127],
        [5.488644168240938, 51.42009670071329],
        [5.488345696419913, 51.420028453490715]
    ],
    [
        [5.501149290906795, 51.4223639138599],
        [5.5009076675460165, 51.42221998989163],
        [5.501679419772273, 51.42169151518314],
        [5.502678370083771, 51.42225597092616],
        [5.50241510881105, 51.42239989478114],
        [5.501765971424646, 51.42195912654381],
        [5.501149290906795, 51.4223639138599]
    ],
    [
        [5.460946434395254, 51.43035395528261],
        [5.4604985090147125, 51.42993147397817],
        [5.461210595517542, 51.42960207906458],
        [5.461662349320193, 51.42941828521862],
        [5.462412720042806, 51.430081849127504],
        [5.462064333635595, 51.43021074171892],
        [5.461788687248202, 51.429991146715935],
        [5.460946434395254, 51.43035395528261]
    ],
    [
        [5.444885035992172, 51.42007169912026],
        [5.44524278164252, 51.41910093602877],
        [5.4476986571833095, 51.420324938274774],
        [5.447398924341599, 51.4205480287485],
        [5.446345024994912, 51.42024052537914],
        [5.44554251448281, 51.42013802379617],
        [5.444885035992172, 51.42007169912026]
    ]
]

# Method for checking if a feature is a mobility hub zone:
# used later in the Folium map to style the hub locations.
def isHub(feature):
    coordinates = feature["geometry"]["coordinates"][0]
    return coordinates in hubCoordinates

# Add the hub locations to the map
folium.GeoJson(
    hubPolygon,
    name="Hub Locations",
    style_function=lambda feature: {
        "fillColor": "red" if isHub(feature) else "blue", 
        "color": "black",
        "weight": 1,
        "fillOpacity": 0.5 if isHub(feature) else 0.2,
    },
).add_to(hubMapEindhoven)
display(HTML(hubMapEindhoven._repr_html_()))

## Measuring improvement and eliminating inefficiencies
Our goal in this part of the modelling assignment is to adjust the placement of the zones such that the efficiency of their placement is maximal. We thus need to define two things for the process of elimination that is to follow: the scoring method and the goal.

1) **Scoring method**: in our development of the original map, we measured the suitability of a point on the overlaid grid to fit a 15-minute city by measuring its adherence to the walkability-standard of the city (as measured by the density of services.) To align with this, we make each mobility hub zone an "upgrade" to the grid it overlays: each point within the 15-minute walking radius of a hub has its "accessibility" radius expanded by the size that can be covered within 15 minutes by biking (minus the time spent walking.) This allows for a solid model of how each zone improves the area around it (though one still recquiring solid simulation)

2) **Goal**: as the name of the category implies, this is an "elimination process": after running the model, we will highlight the zones which improve accessibility by the highest margin (as measured by the overall increase in service density from the points affected by the hub placement), and extract them for our later simulation.

We begin by first overlaying the grid onto the map, as we did during our primary simulation:

In [ ]:
import numpy as np
from folium.plugins import MarkerCluster

# TO IMPROVE: This is almost the same as the code for the initial map overlay,
# so it would be ideal to refactor this into a function that takes the polygon and tags as parameters.
### UPDATE: Not really all that similar anymore, due to the issues with developing the buffers.

# We reuse the tags used in the previous map, since they are still relevant.
# The same is true for the grid-generation method.

# Very much similar to the previous code, we will now analyze the hub zones.
# ...Almost copy-paste, actually, but with some modifications to the radius used
# (see description above.)

# WE DON'T NEED TO USE THE HUBGEODATAFRAME
# WE CAN JS

hubScores = []
averageWalkingSpeed = 4  # km/h
averageBikingSpeed = 15  # km/h

walkingRadius = 0.012
bikingRadius = 0.1 # Increased radius for mobility hubs
mobilityHubRadius = 0.005

lowScoreZones = [] # Store low-efficiency points for visualization
highScoreZones = []
maxPoints = 500

# def isInMobilityHub(point, hubCoordinates, radius = 0.005):
#     return any(Point(coord).distance(point) <= radius for coords in hubCoordinates for coord in coords)

def isInMobilityHub(points_gdf, hubCoordinates, radius = mobilityHubRadius):
    all_hub_points = [Point(coord) for coords in hubCoordinates for coord in coords]
    hub_GDF = gpd.GeoDataFrame(geometry=all_hub_points, crs = 'EPSG:4326')
    return points_gdf.geometry.apply(lambda p: any(p.distance(hub_point) <= radius for hub_point in hub_GDF.geometry))

for polygon in polygons:
    print("Processing a hub zone...should be quick.")
    # This time we will only generate the grid of points of amenity access,
    # disregarding the bus stops for now.
    grid_points = generateGrid(polygon)
    print(f"Generated {len(grid_points)} grid points within the hub zone.")
    
    # FIXED: Convert grid points to a geoDataFrame:
    grid_gdf = gpd.GeoDataFrame(geometry=grid_points, crs="EPSG:4326")
    
    # Query OSM for the specified tags:
    
    amenities = ox.features_from_polygon(polygon, tags) #to check if this even works...
    if isinstance(amenities, dict):
        amenities = ox.features.features_from_place(polygon, tags)
    print(f"Found {len(amenities)} amenities in the hub zone.")
    
    # Now, need to identifty the mobility hubs within the hub zone:
    isZone = isInMobilityHub(grid_gdf, hubCoordinates)
    
    # Create buffers around the grid points based on whether they are in a mobility hub or not.
    # Saves time from previous execution method.
    buffers = [
        point.buffer(bikingRadius if is_hub else walkingRadius)
        for point, is_hub in zip(grid_gdf.geometry, isZone)
    ]
    buffers_gdf = gpd.GeoDataFrame(geometry=buffers, crs="EPSG:4326")
 
    # Check accessibility for each grid point, making sure that
    # the isochrone radius is increased for regions containing mobility hubs,
    # as dictated by the buffers we created above.
    # This generally follows the accessibility check provided in the original grid implementation.
    
    amenityCount = []
    for buffer_geometry in buffers_gdf.geometry:
        count = amenities[amenities.intersects(buffer_geometry)].shape[0]
        amenityCount.append(count)
        
    maxAmenityCount = max(amenityCount) if amenityCount else 0
    threshold = maxAmenityCount * 0.3  # Threshold for low-scoring points
    
    lowScoreFlag = [count < threshold for count in amenityCount]
    lowScoringPoints = grid_gdf[lowScoreFlag]
    
    highScoreFlag = [count >= threshold for count in amenityCount]
    highScoringPoints = grid_gdf[highScoreFlag]
    
    print(f"Identified {len(lowScoringPoints)} low-scoring points in the hub zone.")
    
    # Append the list of low scoring zones with the new low-scoring point.
    lowScoreZones.append(lowScoringPoints)
    
    
# Visualization almost the same as before, excluding the bus stops.
lowScoreZonesGDF = gpd.GeoDataFrame(pd.concat(lowScoreZones, ignore_index=True))
highScoreZonesGDF = gpd.GeoDataFrame(pd.concat(highScoreZones, ignore_index=True))

hubMapEindhoven = folium.Map(location=[51.4416, 5.4697], zoom_start=12)
marker_cluster = MarkerCluster().add_to(hubMapEindhoven)

# Add circle markers with a limit for performance
for _, row in lowScoreZonesGDF.head(maxPoints).iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=4,
        color="red",
        fill=True,
        fill_opacity=0.8,
    ).add_to(marker_cluster)

print("Rendering map...")
hubMapEindhoven


Processing a hub zone...should be quick.
Generated 640 grid points within the hub zone.
Found 358 amenities in the hub zone.
Identified 375 low-scoring points in the hub zone.
Processing a hub zone...should be quick.
Generated 499 grid points within the hub zone.
Found 571 amenities in the hub zone.
Identified 238 low-scoring points in the hub zone.
Processing a hub zone...should be quick.
Generated 229 grid points within the hub zone.
Found 103 amenities in the hub zone.
Identified 10 low-scoring points in the hub zone.
Processing a hub zone...should be quick.
Generated 254 grid points within the hub zone.
Found 172 amenities in the hub zone.
Identified 33 low-scoring points in the hub zone.
Processing a hub zone...should be quick.
Generated 183 grid points within the hub zone.
Found 67 amenities in the hub zone.
Identified 4 low-scoring points in the hub zone.
Rendering map...


/Users/macias/anaconda3/envs/ox/lib/python3.12/site-packages/geopandas/array.py:1638: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  return GeometryArray(data, crs=_get_common_crs(to_concat))


## Adapting for simulation